# Feature： Free Shipping Bar with 50$ Threshold

### 1. import Packages and load dataset

In [1]:
import pandas as pd
import math
import scipy.stats
from matplotlib import pyplot as plt
import numpy as np

In [2]:
##import data
# 1.1 AB Category: https://drive.google.com/file/d/1F7-jE-fOPEg-ytnbwG5YfsrbsoHau6u2/view?usp=sharing
# 1.2 AB Rev Data: https://drive.google.com/file/d/1CQjswN9xbipO6x-Ec3MnGcc7eSbEjFCE/view?usp=sharing
# 1.3 AB Visit Data Test1_2: https://drive.google.com/file/d/1L8VRtjmGXfY_3HCJSKYlW1_-dRug5LKJ/view?usp=sharing
df_Test_Overall = pd.read_csv(r'1.3 AB Visit Data Test1_2.csv')
df_rev = pd.read_csv(r'1.2 AB Rev Data.csv')
df_categorymap=pd.read_csv(r'1.1 AB Category Mapping.csv')

In [3]:
df_Test_Overall.head(3)

,Unnamed: 0,testid,ControlGroup,Date,SessionID,ChannelID,CategoryID,PlatformID,CusID,VisitorTypeID,Time,Bounced,SawProduct,AddedToCart,ReachedCheckout,Converted
0,0,2,1,2019-07-11,7685274065188263037,611,3,1,5299256406894404939,4,434,0,1,0,0,0
1,1,2,1,2019-07-11,6449284229458809869,235,1,1,706543536253348169,4,338,0,1,0,0,0
2,2,2,0,2019-07-11,509870060026686560,336,1,1,2082623938737374415,2,0,1,0,0,0,0


### 2. Filter data for test2, and check traffic.

In [4]:
df_Test=df_Test_Overall[df_Test_Overall['testid']==2]
df_Test.shape

(6870668, 16)

#### Test2 has 6,870,668 rows of data(6 million)

In [5]:
#check the number of control and var
df_Test.drop_duplicates(['ControlGroup','SessionID']).groupby(['ControlGroup'])['SessionID'].count()
#yes,they are almost equal!

ControlGroup
0    3436671
1    3433997
Name: SessionID, dtype: int64

In [6]:
#check how many traffic are in our test(2) each day
df_Test.drop_duplicates(['Date','SessionID']).groupby(['Date'])['SessionID'].count()
#and we found out the 4 days at the beginning and the last day our traffic have some problem

Date
2019-06-14        69
2019-06-15     59090
2019-06-16    127364
2019-06-17    137998
2019-06-18    173863
2019-06-19    194491
2019-06-20    193602
2019-06-21    200563
2019-06-22    199843
2019-06-23    193444
2019-06-24    180545
2019-06-25    222073
2019-06-26    266630
2019-06-27    248291
2019-06-28    257540
2019-06-29    252746
2019-06-30    233204
2019-07-01    199913
2019-07-02    242003
2019-07-03    298940
2019-07-04    305270
2019-07-05    669742
2019-07-06    363540
2019-07-07    290989
2019-07-08    239339
2019-07-09    275926
2019-07-10    358101
2019-07-11    332720
2019-07-12    152829
Name: SessionID, dtype: int64

In [7]:
# We want to exclude the date for these days from the original test dataset for test2.
df_Test=df_Test[pd.to_datetime(df_Test['Date'])>pd.to_datetime('2019-06-17')]
df_Test=df_Test[pd.to_datetime(df_Test['Date'])<pd.to_datetime('2019-07-12')]

In [8]:
#let's see how our data looks now
df_Test.drop_duplicates(['Date','SessionID']).groupby(['Date'])['SessionID'].count()
#yeahee!we've exclude them succussfully

Date
2019-06-18    173863
2019-06-19    194491
2019-06-20    193602
2019-06-21    200563
2019-06-22    199843
2019-06-23    193444
2019-06-24    180545
2019-06-25    222073
2019-06-26    266630
2019-06-27    248291
2019-06-28    257540
2019-06-29    252746
2019-06-30    233204
2019-07-01    199913
2019-07-02    242003
2019-07-03    298940
2019-07-04    305270
2019-07-05    669742
2019-07-06    363540
2019-07-07    290989
2019-07-08    239339
2019-07-09    275926
2019-07-10    358101
2019-07-11    332720
Name: SessionID, dtype: int64

In [9]:
import gc
gc.collect()


40

### 3.1 Analyze metrics

#### Count the customer behavior on session level, but we will automate this part with customer level later.

In [10]:
#get the counts of session_id for each test group and store the result into a new pandas dataframe named df_result
df_Result=pd.DataFrame(df_Test.drop_duplicates(['SessionID','ControlGroup']).groupby('ControlGroup')['SessionID'].count())

In [11]:
df_Result

,SessionID
ControlGroup,
0,3197663
1,3195655


In [12]:
metrics=['Bounced','SawProduct','AddedToCart','ReachedCheckout','Converted']
#create loop to get all the counts of "1" for each metrics
for metric in metrics:
    #get the column of "1s" for that metric,then count the number of it,then store the row of result number as a.
    a=df_Test[df_Test[metric]==1].drop_duplicates(['SessionID','ControlGroup']).groupby('ControlGroup')['SessionID'].count()
    #define the column name as that metric name
    a.name=metric
    #join each result on the df_result dataframe we've crested above.
    df_Result=df_Result.join(a)

In [13]:
df_Result

,SessionID,Bounced,SawProduct,AddedToCart,ReachedCheckout,Converted
ControlGroup,,,,,,
0,3197663,632191,2325995,975934,483993,354340
1,3195655,631957,2325654,975034,483729,353260


#### Create a z_test calculation tool

In [14]:
def z_test_calculator(df,denominator,numerator):
    #df: dataframe containing the result (absolute counts)
    #denominator: str
    #numerator:str
    
    #get the data [1,]or[0,] is control or variation,denominator and numerator is our imput,
    #normally it should be a count of total number, and a count of the metric number we observed.
    control_denominator=df.loc[1,denominator]
    var_denominator=df.loc[0,denominator]
    control_numerator=df.loc[1,numerator]
    var_numerator=df.loc[0,numerator]    
    
    #caculate the Rate: simply do 2 divide
    control_rate=control_numerator/control_denominator
#     print(control_numerator)
#     print(control_denominator)
    var_rate=var_numerator/var_denominator
    
    #calculate the STD: sd=p(1-p)/N 再开根号
#     print(control_rate)
    control_sd=math.sqrt(control_rate*(1-control_rate)/control_denominator)
    
    var_sd=math.sqrt(var_rate*(1-var_rate)/var_denominator)
    
    #z score =两个rate的差值/control和variation的平均标准差
    #control和variation的平均标准差=control的标准差的平方+variation的标准差的平方 再开根号
    
    z_score=(control_rate-var_rate)/math.sqrt(pow(control_sd,2)+pow(var_sd,2))
    
    
    #p value python有方法可以直接找到pvalue，simply input the absolute value of z_score.
    p_value=scipy.stats.norm.sf(abs(z_score))
    
    
    #lift simple subtract and divide
    perc_lift=(var_rate-control_rate)/control_rate
    abs_lift=(var_rate-control_rate)
    
    return (p_value,perc_lift,abs_lift)

#### Real Calculation part, calculate all metrics on both levels together

In [15]:
user_types=['SessionID','CusID']
dic_final={}
metrics=['Bounced','SawProduct','AddedToCart','ReachedCheckout','Converted']

for user_type in user_types:
    df_Result_any=pd.DataFrame(df_Test.drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count())
    
    for metric in metrics:
        a=df_Test[df_Test[metric]==1].drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count()
        a.name=metric
        df_Result_any=df_Result_any.join(a)

    KPIs=[(user_type,'Bounced'),
          (user_type,'SawProduct'),
          (user_type,'AddedToCart'),
          (user_type,'ReachedCheckout'),
          (user_type,'Converted'),
          ('AddedToCart','ReachedCheckout'),
          ('ReachedCheckout','Converted'),
          ('AddedToCart','Converted')]

    for index in df_Result_any.index:
        j=0
        if index!=1:
            df_each_group=df_Result_any

            df_final=pd.DataFrame()

            for i in KPIs:
                result=z_test_calculator(df_each_group,i[0],i[1])
                df_final.loc[j,'denominator']=i[0]
                df_final.loc[j,'numerator']=i[1]
                df_final.loc[j,'p_value']=result[0]
                df_final.loc[j,'perc_lift']=result[1]
                df_final.loc[j,'abs_lift']=result[2]
                j=j+1
            dic_final[user_type]=df_final


In [16]:
#check what does df_Result_any look like now, should be for customer level now, because it's the last item in user_types list.
df_Result_any

,CusID,Bounced,SawProduct,AddedToCart,ReachedCheckout,Converted
ControlGroup,,,,,,
0,859851,311019,810566,669829,387717,313232
1,859769,310735,810408,669890,387516,312617


In [17]:
#check the value for key "CusID", it should be results for customer level
dic_final['CusID']

,denominator,numerator,p_value,perc_lift,abs_lift
0,CusID,Bounced,0.343214,0.000819,0.000296
1,CusID,SawProduct,0.395637,0.000100,0.000094
2,CusID,AddedToCart,0.409220,-0.000186,-0.000145
3,CusID,ReachedCheckout,0.400754,0.000423,0.000191
4,CusID,Converted,0.176847,0.001872,0.000681
5,AddedToCart,ReachedCheckout,0.339638,0.000610,0.000353
6,ReachedCheckout,Converted,0.096172,0.001448,0.001168
7,AddedToCart,Converted,0.132572,0.002059,0.000961


#### Export result into excel file, in multiple sheets.

In [18]:
writer = pd.ExcelWriter('3.1 Final_data_Test2.xlsx')
for key in dic_final.keys():
    dic_final[key].to_excel(writer, sheet_name=key)
writer.save()

#### Re-calculate everything in different cuts(different categories in item purchased, platform using, and visitor type)

In [19]:
df_test_data = df_Test.copy()
dic_final_cuts_usertype = {}
user_types = ['SessionID','CusID']
cuts = ['CategoryID', 'PlatformID','VisitorTypeID']

In [20]:
for user_type in user_types:
    print(user_type)
    for cut in cuts:
        print(cut)
 
        for p in set(df_test_data[cut]):

            df_Test_cut = df_test_data[df_test_data[cut]==p]
            df_Result_any1 = pd.DataFrame(df_Test_cut.drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count())

            metrics = ['Bounced','SawProduct','AddedToCart','ReachedCheckout','Converted']
            for metric in metrics:
                a = df_Test_cut[df_Test_cut[metric]==1].drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count()
                a.name = metric
                df_Result_any1 = df_Result_any1.join(a)
                print(df_Result_any1)


            KPIs=[(user_type,'Bounced'),
          (user_type,'SawProduct'),
          (user_type,'AddedToCart'),
          (user_type,'ReachedCheckout'),
          (user_type,'Converted'),
          ('AddedToCart','ReachedCheckout'),
          ('ReachedCheckout','Converted'),
          ('AddedToCart','Converted')]

            for index in df_Result_any1.index:
                j=0
                #reset the index
                if index!=1:
                    df_each_group = df_Result_any1
                    df_final=pd.DataFrame()


                    for i in KPIs:
                        result=z_test_calculator(df_each_group,i[0],i[1])
                        df_final.loc[j,'denominator']=i[0]
                        df_final.loc[j,'numerator'] = i[1]
                        df_final.loc[j,'p_value'] = result[0]
                        df_final.loc[j,'perc_lift'] = result[1]
                        df_final.loc[j,'abs_lift']= result[2]
                        j=j+1

                    dic_final_cuts_usertype[user_type+'_'+cut+str(p)]= df_final 

SessionID
CategoryID
              SessionID  Bounced
ControlGroup                    
0               2065904   337552
1               2063830   337170
              SessionID  Bounced  SawProduct
ControlGroup                                
0               2065904   337552     1309013
1               2063830   337170     1307975
              SessionID  Bounced  SawProduct  AddedToCart
ControlGroup                                             
0               2065904   337552     1309013       647669
1               2063830   337170     1307975       646297
              SessionID  Bounced  SawProduct  AddedToCart  ReachedCheckout
ControlGroup                                                              
0               2065904   337552     1309013       647669           343608
1               2063830   337170     1307975       646297           343107
              SessionID  Bounced  SawProduct  AddedToCart  ReachedCheckout  \
ControlGroup                                             

              SessionID  Bounced
ControlGroup                    
0                301076    49405
1                300798    49258
              SessionID  Bounced  SawProduct
ControlGroup                                
0                301076    49405      250431
1                300798    49258      249812
              SessionID  Bounced  SawProduct  AddedToCart
ControlGroup                                             
0                301076    49405      250431       133756
1                300798    49258      249812       134197
              SessionID  Bounced  SawProduct  AddedToCart  ReachedCheckout
ControlGroup                                                              
0                301076    49405      250431       133756            55606
1                300798    49258      249812       134197            55803
              SessionID  Bounced  SawProduct  AddedToCart  ReachedCheckout  \
ControlGroup                                                                 


               CusID  Bounced  SawProduct  AddedToCart  ReachedCheckout  \
ControlGroup                                                              
0             105881    35985       97790        43318            19947   
1             105770    35495       97902        43841            20192   

              Converted  
ControlGroup             
0                 14153  
1                 14395  
               CusID  Bounced
ControlGroup                 
0             215162    71613
1             214513    71613
               CusID  Bounced  SawProduct
ControlGroup                             
0             215162    71613      198412
1             214513    71613      197984
               CusID  Bounced  SawProduct  AddedToCart
ControlGroup                                          
0             215162    71613      198412       100668
1             214513    71613      197984       100532
               CusID  Bounced  SawProduct  AddedToCart  ReachedCheckout
ControlGroup   

In [21]:
dic_final_cuts_usertype['SessionID_CategoryID2']

,denominator,numerator,p_value,perc_lift,abs_lift
0,SessionID,Bounced,0.158042,-0.004338,-0.001111
1,SessionID,SawProduct,0.269255,-0.000548,-0.000488
2,SessionID,AddedToCart,0.178317,-0.003555,-0.001075
3,SessionID,ReachedCheckout,0.371294,-0.002131,-0.000283
4,SessionID,Converted,0.215666,0.006026,0.000599
5,AddedToCart,ReachedCheckout,0.392176,0.001430,0.000628
6,ReachedCheckout,Converted,0.021285,0.008174,0.006115
7,AddedToCart,Converted,0.073090,0.009616,0.003160


#### Export result into excel file, in multiple sheets.

In [22]:
#excel contain all the possible cuts
writer = pd.ExcelWriter('3.2 Final_cuts_data_Test2.xlsx')
for key in dic_final_cuts_usertype.keys():
    dic_final_cuts_usertype[key].to_excel(writer, sheet_name=key)
writer.save()

### 3.2 Analyze revenue

In [23]:
df_cusid=df_Test_Overall[['SessionID','ControlGroup','CusID','CategoryID','VisitorTypeID']]
df_cusid.head(3)

,SessionID,ControlGroup,CusID,CategoryID,VisitorTypeID
0,7685274065188263037,1,5299256406894404939,3,4
1,6449284229458809869,1,706543536253348169,1,4
2,509870060026686560,0,2082623938737374415,1,2


In [24]:
df_Rev=pd.merge(df_rev,df_cusid, on = ['SessionID','ControlGroup'])

In [25]:
df_Rev=df_Rev[df_Rev['testid']==2]
df_Rev.head(3)

,testid,ControlGroup,"""Date""",SessionID,TotalRevenue,CusID,CategoryID,VisitorTypeID
0,2,1,2019-06-26,3170596292771148233,57.98,6955488039014672215,1,4
1,2,1,2019-06-26,4521151331680385048,58.12,6415996569600686642,1,4
2,2,1,2019-06-26,4428930296371807923,383.65,3331805327447778683,1,3


In [26]:
df_Rev.drop_duplicates(['ControlGroup','SessionID']).groupby(['ControlGroup'])['SessionID'].count()

ControlGroup
0    386723
1    385706
Name: SessionID, dtype: int64

In [27]:
Control_Rev=df_Rev[df_Rev['ControlGroup']==1]['TotalRevenue'].array
Control_Rev
Var_Rev=df_Rev[df_Rev['ControlGroup']==0]['TotalRevenue'].array
Control_Rev
Var_Rev

<PandasArray>
[ 178.19,   84.49,  331.94,  109.99,   149.9,   55.09,   79.98,   56.99,
  150.95,  869.38,
 ...
  143.11,  138.99,  637.95, 1885.99,   72.99,  111.99, 2395.96,   32.98,
   92.97,   33.98]
Length: 386723, dtype: float64

In [28]:
P1=np.percentile(Control_Rev,95)
P2=np.percentile(Var_Rev,95)

In [29]:
scipy.stats.mannwhitneyu(Control_Rev,Var_Rev)

MannwhitneyuResult(statistic=74548964432.0, pvalue=0.37305166491932945)

In [30]:
def z_test_calculator_continuous(df,denominator,numerator,numerator_sq):
    #df: dataframe containing the result (absolute counts)
    #denominator: str
    #numerator:str
    #numerator_sq:str
    
    #get the data
    control_denominator=df.loc[1,denominator]
    var_denominator=df.loc[0,denominator]
    control_numerator=df.loc[1,numerator]
    var_numerator=df.loc[0,numerator]    
    
    #Rate
    control_rate=control_numerator/control_denominator
    var_rate=var_numerator/var_denominator
    
    #Variance

    control_var=df.loc[1,numerator_sq]/control_denominator-control_rate**2
    var_var=df.loc[0,numerator_sq]/var_denominator-var_rate**2
    
    #z score
    z_score=(control_rate-var_rate)/math.sqrt(control_var/control_denominator+var_var/var_denominator)
    
    
    #p value
    p_value=scipy.stats.norm.sf(abs(z_score))
    
    
    #lift
    perc_lift=(var_rate-control_rate)/control_rate
    abs_lift=(var_rate-control_rate)
    
    return (p_value,perc_lift,abs_lift)

In [31]:
user_types=['SessionID','CusID']
dic_rev_final={}


for user_type in user_types:
    print(user_type)
    df_Result = pd.DataFrame(df_Rev.drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count())
              
    df_Result.loc[1,'Rev']=sum(Control_Rev[Control_Rev<P1])
    df_Result.loc[0,'Rev']=sum(Var_Rev[Var_Rev<P2])

    df_Result.loc[1,'Rev_sq']=sum(Control_Rev[Control_Rev<P1]**2)
    df_Result.loc[0,'Rev_sq']=sum(Var_Rev[Var_Rev<P2]**2)
    
    for index in df_Result.index:
    
        if index!=1:
            df_each_group=df_Result
            df_final=pd.DataFrame()
            result=z_test_calculator_continuous(df_each_group,user_type,'Rev','Rev_sq')

            dic_rev_final[user_type]= result

SessionID
CusID


In [32]:
dic_rev_final

{'SessionID': (0.47943916811101006,
  -0.00010956668021356469,
  -0.022994679611826996),
 'CusID': (0.2831296544127835, 0.001119122817330874, 0.26705630482015863)}

In [33]:
dic_rev_final=pd.DataFrame(dic_rev_final).T
dic_rev_final=dic_rev_final.rename(columns={0:'p_value',1:'perc_lift',2:'abs_lift'})
dic_rev_final

,p_value,perc_lift,abs_lift
SessionID,0.479439,-0.000110,-0.022995
CusID,0.283130,0.001119,0.267056


In [34]:
writer = pd.ExcelWriter('3.3 Final_revenue_data_Test2.xlsx')
dic_rev_final.to_excel(writer, sheet_name='revenue_data_Test2')
writer.save()

In [35]:
df_rev_data = df_Rev.copy()
dic_rev_final_cuts_usertype = {}
user_types = ['SessionID','CusID']
cuts = ['CategoryID','VisitorTypeID']

In [36]:
for user_type in user_types:
    print(user_type)
    for cut in cuts:
        print(cut)
 
        for p in set(df_rev_data[cut]):

            df_Rev_cut = df_rev_data[df_rev_data[cut]==p]
            Control_Rev_cut=df_Rev_cut[df_Rev_cut['ControlGroup']==1]['TotalRevenue'].array
            Var_Rev_cut=df_Rev_cut[df_Rev_cut['ControlGroup']==0]['TotalRevenue'].array
           
            df_Result_any = pd.DataFrame(df_Rev_cut.drop_duplicates([user_type,'ControlGroup']).groupby('ControlGroup')[user_type].count())
            
            df_Result_any.loc[1,'Rev']=sum(Control_Rev_cut)
            df_Result_any.loc[0,'Rev']=sum(Var_Rev_cut)
            df_Result_any.loc[1,'Rev_sq']=sum(Control_Rev_cut**2)
            df_Result_any.loc[0,'Rev_sq']=sum(Var_Rev_cut**2)

            for index in df_Result.index:

                if index!=1:
                    df_each_group=df_Result_any
                    df_final=pd.DataFrame()
                    result=z_test_calculator_continuous(df_each_group,user_type,'Rev','Rev_sq')

                    dic_rev_final_cuts_usertype [user_type+'_'+cut+str(p)]= result

SessionID
CategoryID
VisitorTypeID
CusID
CategoryID
VisitorTypeID


In [37]:
dic_rev_final_cuts_usertype

{'SessionID_CategoryID1': (0.1669513039029697,
  -0.004007124355840233,
  -1.2190637916609148),
 'SessionID_CategoryID2': (0.13828786389951014,
  -0.012429797836869354,
  -3.87182226136332),
 'SessionID_CategoryID3': (0.18811397157802334,
  -0.012917244256377754,
  -3.67487281400048),
 'SessionID_CategoryID4': (0.04525062502964287,
  -0.0256380939650991,
  -10.138105154206073),
 'SessionID_CategoryID5': (0.08616456625346552,
  0.015975706662562737,
  3.5756524436920927),
 'SessionID_VisitorTypeID1': (0.362676104205879,
  0.005726998795823715,
  1.3826202471863667),
 'SessionID_VisitorTypeID2': (0.13891434473102615,
  0.010722936381297756,
  3.208049489918835),
 'SessionID_VisitorTypeID3': (0.28078399133248366,
  -0.0042619821416727335,
  -1.381876900241764),
 'SessionID_VisitorTypeID4': (0.026697348835632823,
  -0.008650006278191963,
  -2.5486193107296344),
 'CusID_CategoryID1': (0.3264070253001219,
  -0.0018234146574515806,
  -0.6152165643505896),
 'CusID_CategoryID2': (0.084247171920

In [38]:
dic_rev_final_cuts_usertype=pd.DataFrame(dic_rev_final_cuts_usertype).T
dic_rev_final_cuts_usertype=dic_rev_final_cuts_usertype.rename(columns={0:'p_value',1:'perc_lift',2:'abs_lift'})
dic_rev_final_cuts_usertype

,p_value,perc_lift,abs_lift
SessionID_CategoryID1,0.166951,-0.004007,-1.219064
SessionID_CategoryID2,0.138288,-0.012430,-3.871822
SessionID_CategoryID3,0.188114,-0.012917,-3.674873
SessionID_CategoryID4,0.045251,-0.025638,-10.138105
SessionID_CategoryID5,0.086165,0.015976,3.575652
SessionID_VisitorTypeID1,0.362676,0.005727,1.382620
SessionID_VisitorTypeID2,0.138914,0.010723,3.208049
SessionID_VisitorTypeID3,0.280784,-0.004262,-1.381877
SessionID_VisitorTypeID4,0.026697,-0.008650,-2.548619
CusID_CategoryID1,0.326407,-0.001823,-0.615217


In [39]:
writer = pd.ExcelWriter('3.4 Final_revenue_cuts_data_Test2.xlsx')
dic_rev_final_cuts_usertype.to_excel(writer, sheet_name='revenue_cuts_data_Test2')
writer.save()